In [ ]:
import requests
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier
from sklearn.feature_selection import SelectKBest, chi2,SelectFwe

In [ ]:
api_key = '5ea597c0ccd249bf931ae9b5ee31ba42' 

url = 'https://api.sportsdata.io/v3/nfl/stats/json/PlayerGameStatsByWeek/2022REG/1?' 
headers = {'Ocp-Apim-Subscription-Key': '{key}'.format(key=api_key)}
players = requests.get(url, headers=headers).json() 
    
url = 'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/2022REG/1?' 
headers = {'Ocp-Apim-Subscription-Key': '{key}'.format(key=api_key)}
scores = requests.get(url, headers=headers).json() 
    
players = pd.DataFrame(players)
scores = pd.DataFrame(scores)

#for i in players.columns:
    #print(i)
    
scores = scores[["HomeScore","AwayScore","HomeTeam","AwayTeam"]]
#players = players[["Team","Played","PassingYards","RushingYards","ReceivingYards","PuntReturnYards","KickReturnYards"]]
players = players[players["Played"]==1] #only include players who played
players = players.groupby("Team").sum()
    
win = []
for i in range(len(players)):
    team = str(players.index[i])
    if sum(scores["HomeTeam"].str.contains(team))==1:
        curr = scores[scores["HomeTeam"]==team]
        if curr["HomeScore"].iloc[0] > curr["AwayScore"].iloc[0]:
            win.append(1)
        else:
            win.append(0)
    elif sum(scores["AwayTeam"].str.contains(team))==1:
        curr = scores[scores["AwayTeam"]==team]
        if curr["HomeScore"].iloc[0] < curr["AwayScore"].iloc[0]:
            win.append(1)
        else:
            win.append(0)
                
players["Win"] = win
print("Done")

In [ ]:
print("Week 1")
print("Size:", len(players))
n = 16
for week in range(2,n+1):
    print("Week", week)

    url = 'https://api.sportsdata.io/v3/nfl/stats/json/PlayerGameStatsByWeek/2022REG/'+str(week)+'?'
    headers = {'Ocp-Apim-Subscription-Key': '{key}'.format(key=api_key)}
    players_curr = requests.get(url, headers=headers).json() 
    
    url = 'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/2022REG/'+str(week)+'?' 
    headers = {'Ocp-Apim-Subscription-Key': '{key}'.format(key=api_key)}
    scores_curr = requests.get(url, headers=headers).json() 

    players_curr = pd.DataFrame(players_curr)
    scores_curr = pd.DataFrame(scores_curr)

    scores_curr = scores_curr[["HomeScore","AwayScore","HomeTeam","AwayTeam"]]
    #players_curr = players_curr[["Team","Played","PassingYards","RushingYards","ReceivingYards","PuntReturnYards","KickReturnYards"]]
    players_curr = players_curr[players_curr["Played"]==1] #only include players who played
    players_curr = players_curr.groupby("Team").sum()

    win_curr = []
    for i in range(len(players_curr)):
        team = str(players_curr.index[i])
        if sum(scores_curr["HomeTeam"].str.contains(team))==1:
            curr = scores_curr[scores_curr["HomeTeam"]==team]
            if curr["HomeScore"].iloc[0] > curr["AwayScore"].iloc[0]:
                win_curr.append(1)
            else:
                win_curr.append(0)
        elif sum(scores_curr["AwayTeam"].str.contains(team))==1:
            curr = scores_curr[scores_curr["AwayTeam"]==team]
            if curr["HomeScore"].iloc[0] < curr["AwayScore"].iloc[0]:
                win_curr.append(1)
            else:
                win_curr.append(0)

    players_curr["Win"] = win_curr
    players = pd.concat([players,players_curr])
    
    print("New size:", len(players))
    if week==n:
        print("Done")

In [ ]:
X = players.drop(["Played","Win","RushingTouchdowns","ExtraPointsMade","OffensiveTouchdowns","Touchdowns","FumbleReturnTouchdowns"],axis=1)
y = players["Win"]
X.dropna(axis=1,inplace=True)
X = X.loc[:,X.apply(pd.Series.nunique) != 1]


In [ ]:
X

In [ ]:
kbest = SelectKBest()
kbest.fit_transform(X, y)
features = kbest.get_feature_names_out()
X = X[features]
features

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def transform(X):
    scaler = StandardScaler()
    scaler.fit(X)
    X_transform = scaler.transform(X)
    pca=PCA(n_components=2)
    pca.fit(X_transform)
    X_pca=pca.transform(X_transform)
    return X_pca

In [ ]:
#X_train_pca=transform(X_train)
#X_test_pca = transform(X_test)

In [ ]:
#plt.scatter(X_train_pca[:,0],X_train_pca[:,1],c=y_train)

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))

In [ ]:
pipeline_optimizer = TPOTClassifier(generations=10)

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))